In [1]:
import asyncio
import json
import dotenv
import os
import requests as rq
import pandas as pd
import numpy as np
from tqdm import notebook, tqdm
from sqlalchemy import create_engine
from concurrent.futures import ThreadPoolExecutor

pd.set_option('max.columns', 500)

### Load env and Create DB Connection

- Create and environment file with PASSWORD, USERNAME and TEAM_ID
- Initialize the SQLite DB Connection

In [5]:
dotenv.load_dotenv()

db_conn = create_engine('sqlite:///fpl_data_sqlite.db')

### Declare Constants

In [19]:
START_GW = 1

CURRENT_SEASON = '2021/22'

GW_LOOKAHEAD_NUM = 3

NEW_SEASON = False

SAVE_TO_DB_FLAG = True

### Fetch Base Info
This is the base api which has the following information:
- info about all the teams (teams)
- player information and stats till the current gameweek (elements)
- info about all gameweeks in current season (events)

In [14]:
base_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

base_url_response = rq.request('GET', base_url)
base_url_response_dict = json.loads(base_url_response.text)

In [15]:
teams_info_df = pd.DataFrame(base_url_response_dict['teams'])

In [25]:
if SAVE_TO_DB_FLAG:
    teams_info_df.to_sql('teams_info', db_conn, if_exists='replace', index=False)

### Calculate Next Gameweek No

In [17]:
next_gameweek = min([i['id'] for i in base_url_response_dict['events'] if i['finished'] == False])
next_gameweek

11

In [12]:
current_max_gw = pd.read_sql("select max(calculated_at_gw) as max_gw from cummumative_player_info", db_conn)['max_gw'][0]
current_max_gw

10

In [20]:
if current_max_gw == next_gameweek - 1:
    SAVE_TO_DB_FLAG = False

### Fetch Fixtures Info
This api provides information at a fixure and gameweek level for all gameweeks (Both Past and Future Gameweeks)

In [8]:
fixt_url = 'https://fantasy.premierleague.com/api/fixtures/'

fixt_response = rq.request('GET', fixt_url)
fixt_response_dict = json.loads(fixt_response.text)

In [9]:
fixtures_df = pd.DataFrame(fixt_response_dict)

In [10]:
fixtures_df['season'] = CURRENT_SEASON
fixtures_df.drop('stats', axis=1, inplace=True, errors='ignore')
fixtures_df['kickoff_time'] = pd.to_datetime(fixtures_df['kickoff_time'])

In [11]:
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id,season
0,2210271,1,True,True,1,2021-08-13 19:00:00+00:00,90,False,True,1,0.0,3,2.0,3,2,66342,2021/22
1,2210276,1,True,True,6,2021-08-14 11:30:00+00:00,90,False,True,10,1.0,13,5.0,2,4,66347,2021/22
2,2210272,1,True,True,2,2021-08-14 14:00:00+00:00,90,False,True,4,2.0,5,1.0,3,2,66343,2021/22
3,2210273,1,True,True,3,2021-08-14 14:00:00+00:00,90,False,True,7,0.0,6,3.0,2,5,66344,2021/22
4,2210274,1,True,True,4,2021-08-14 14:00:00+00:00,90,False,True,16,1.0,8,3.0,2,4,66345,2021/22


In [26]:
if SAVE_TO_DB_FLAG:
    fixtures_df.to_sql('fixtures_history_' + CURRENT_SEASON, db_conn, if_exists='replace', index=False)

### Get Next 3 Fixtures and Create a Pivot Table
Perform some data transformation to get info for the next 3 gameweeks based on the current gameweek  
This transformed data will be useful for using as variables while trying to train ML Models or building Optimization Models

In [13]:
next_fixt = fixtures_df[fixtures_df.event.isin(range(next_gameweek, next_gameweek + GW_LOOKAHEAD_NUM))]

In [14]:
next_fixt.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id,season
100,2210379,11,False,False,109,2021-11-05 20:00:00+00:00,0,False,False,2,NaN,16,NaN,3,2,66450,2021/22
101,2210378,11,False,False,108,2021-11-06 12:30:00+00:00,0,False,False,12,NaN,13,NaN,4,4,66449,2021/22
102,2210372,11,False,False,102,2021-11-06 15:00:00+00:00,0,False,False,15,NaN,3,NaN,2,2,66443,2021/22
103,2210374,11,False,False,104,2021-11-06 15:00:00+00:00,0,False,False,5,NaN,6,NaN,2,5,66445,2021/22
104,2210375,11,False,False,105,2021-11-06 15:00:00+00:00,0,False,False,20,NaN,7,NaN,2,2,66446,2021/22


In [15]:
next_fixt_away = next_fixt[['event', 'team_a', 'team_a_difficulty', 'team_h', 'team_h_difficulty']].copy()
next_fixt_away.rename(
    columns = {
        'event':'gw', 'team_a':'team_id', 
        'team_a_difficulty':'match_difficulty',
        'team_h':'opponent_team_id',
        'team_h_difficulty':'match_difficulty_for_opponent'
    },
    inplace=True
)
next_fixt_away['game_type'] = 'away'


next_fixt_home = next_fixt[['event', 'team_a', 'team_a_difficulty', 'team_h', 'team_h_difficulty']].copy()
next_fixt_home.rename(
    columns = {
        'event':'gw', 'team_h':'team_id', 
        'team_h_difficulty':'match_difficulty',
        'team_a':'opponent_team_id',
        'team_a_difficulty':'match_difficulty_for_opponent'
    },
    inplace=True
)
next_fixt_home['game_type'] = 'home'

In [16]:
next_fixt_base = pd.concat([next_fixt_away, next_fixt_home]).reset_index(drop=True)
next_fixt_base = next_fixt_base.pivot(
    index = ['team_id'],
    columns = ['gw'],
    values = ['match_difficulty', 'opponent_team_id', 'match_difficulty_for_opponent', 'game_type']
)
temp = next_fixt_base.columns
next_fixt_base.columns = ['next_' + str( int(i[1]) - (next_gameweek - 1) ) + '_gw_' + i[0] for i in temp]
next_fixt_base = next_fixt_base.reset_index()
next_fixt_base['calculated_at_gw'] = next_gameweek - 1
next_fixt_base['season'] = CURRENT_SEASON

In [17]:
next_fixt_base.head()

,team_id,next_1_gw_match_difficulty,next_2_gw_match_difficulty,next_3_gw_match_difficulty,next_1_gw_opponent_team_id,next_2_gw_opponent_team_id,next_3_gw_opponent_team_id,next_1_gw_match_difficulty_for_opponent,next_2_gw_match_difficulty_for_opponent,next_3_gw_match_difficulty_for_opponent,next_1_gw_game_type,next_2_gw_game_type,next_3_gw_game_type,calculated_at_gw,season
0,1,2,5,2,18,11,14,4,3,4,home,away,home,10,2021/22
1,2,2,3,2,16,4,7,3,3,3,away,home,away,10,2021/22
2,3,2,2,3,15,14,8,2,2,2,home,away,home,10,2021/22
3,4,2,3,2,14,2,10,3,3,3,home,away,home,10,2021/22
4,5,5,2,3,6,7,17,2,2,2,away,home,home,10,2021/22


In [21]:
if SAVE_TO_DB_FLAG:
    next_fixt_base.to_sql('next_three_fixtures_info', db_conn, if_exists='append', index=False)

### Get Past GW Player Stats
This api provides player level stats for all game weeks (stats). It returns information at player id level.   
Need to make individual api calls for each game week

In [19]:
past_gw_stats_base = []

for gw in tqdm(range(START_GW, next_gameweek)):
    player_gw_stats = []
    
    url = 'https://fantasy.premierleague.com/api/event/' + str(gw) + '/live/'

    past_gw_response = rq.request('GET', url)
    past_gw_response = json.loads(past_gw_response.text)

    for item in past_gw_response['elements']:
        item['stats']['player_id'] = item['id'] 
        player_gw_stats.append(item['stats'])
        
    player_gw_stats = pd.DataFrame(player_gw_stats)
    player_gw_stats['gw'] = gw
    player_gw_stats['season'] = CURRENT_SEASON
    
    past_gw_stats_base.append(player_gw_stats)

100%|██████████| 10/10 [00:03<00:00,  3.21it/s]


In [20]:
past_gw_player_stats = pd.concat(past_gw_stats_base).reset_index(drop=True)
past_gw_player_stats.head()

,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,player_id,gw,season
0,90,0,0,0,2,0,0,0,0,0,1,0,11,11.8,0.0,0.0,1.2,1,False,1,1,2021/22
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,False,2,1,2021/22
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,False,3,1,2021/22
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,False,4,1,2021/22
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,False,5,1,2021/22


In [27]:
if SAVE_TO_DB_FLAG:
    past_gw_player_stats.to_sql('player_stats_all_gameweeks_season_'+CURRENT_SEASON, db_conn, if_exists='replace', index=False)

Perform Data Transformation to Get Past 3 Gameweek Player Stats at Player ID Level. Will be useful for building models and further analysis on the data

In [22]:
last_3_gw_stats = past_gw_player_stats[past_gw_player_stats.gw.isin(range(next_gameweek-3, next_gameweek))]
last_3_gw_stats = last_3_gw_stats.pivot(
    index = 'player_id',
    columns = 'gw',
    values = [
        'minutes', 'goals_scored', 'assists', 'clean_sheets', 'saves', 'bonus', 'influence',
        'creativity', 'threat', 'ict_index', 'total_points'
    ]
)

temp = last_3_gw_stats.columns
last_3_gw_stats.columns = ['t_minus_' + str((next_gameweek-1) - int(i[1])) + '_gw_' + i[0] for i in temp]
last_3_gw_stats = last_3_gw_stats.reset_index()
last_3_gw_stats['calculated_at_gw'] = next_gameweek - 1
last_3_gw_stats['season'] = CURRENT_SEASON

In [23]:
last_3_gw_stats.head()

,player_id,t_minus_2_gw_minutes,t_minus_1_gw_minutes,t_minus_0_gw_minutes,t_minus_2_gw_goals_scored,t_minus_1_gw_goals_scored,t_minus_0_gw_goals_scored,t_minus_2_gw_assists,t_minus_1_gw_assists,t_minus_0_gw_assists,t_minus_2_gw_clean_sheets,t_minus_1_gw_clean_sheets,t_minus_0_gw_clean_sheets,t_minus_2_gw_saves,t_minus_1_gw_saves,t_minus_0_gw_saves,t_minus_2_gw_bonus,t_minus_1_gw_bonus,t_minus_0_gw_bonus,t_minus_2_gw_influence,t_minus_1_gw_influence,t_minus_0_gw_influence,t_minus_2_gw_creativity,t_minus_1_gw_creativity,t_minus_0_gw_creativity,t_minus_2_gw_threat,t_minus_1_gw_threat,t_minus_0_gw_threat,t_minus_2_gw_ict_index,t_minus_1_gw_ict_index,t_minus_0_gw_ict_index,t_minus_2_gw_total_points,t_minus_1_gw_total_points,t_minus_0_gw_total_points,calculated_at_gw,season
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,10,2021/22
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,10,2021/22
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,10,2021/22
3,4,90,90,90,1,1,0,0,1,0,0,0,1,0,0,0,3,0,0,40.0,56.0,15.8,15.8,24.6,3.0,35.0,68.0,52.0,9.1,14.9,7.1,9,6,2,10,2021/22
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,10,2021/22


In [22]:
if SAVE_TO_DB_FLAG:
    last_3_gw_stats.to_sql('last_3_gameweeks_player_stats', db_conn, if_exists='append', index=False)

### Cleaning Required Data and Subsetting Columns

Clean up the Player Information fetched from the Base API (elements property)  
This data will serve as the base for most models built on top of this data

In [25]:
elements_list = base_url_response_dict['elements']

In [28]:
### sample
[i for i in elements_list if i['second_name'] == 'Salah']

[{'chance_of_playing_next_round': None,
  'chance_of_playing_this_round': None,
  'code': 118748,
  'cost_change_event': 0,
  'cost_change_event_fall': 0,
  'cost_change_start': 4,
  'cost_change_start_fall': -4,
  'dreamteam_count': 5,
  'element_type': 3,
  'ep_next': '14.5',
  'ep_this': '15.0',
  'event_points': 5,
  'first_name': 'Mohamed',
  'form': '14.0',
  'id': 233,
  'in_dreamteam': True,
  'news': '',
  'news_added': None,
  'now_cost': 129,
  'photo': '118748.jpg',
  'points_per_game': '11.2',
  'second_name': 'Salah',
  'selected_by_percent': '71.0',
  'special': False,
  'squad_number': None,
  'status': 'a',
  'team': 11,
  'team_code': 14,
  'total_points': 112,
  'transfers_in': 2370319,
  'transfers_in_event': 52620,
  'transfers_out': 771937,
  'transfers_out_event': 7562,
  'value_form': '1.1',
  'value_season': '8.7',
  'web_name': 'Salah',
  'minutes': 900,
  'goals_scored': 10,
  'assists': 7,
  'clean_sheets': 6,
  'goals_conceded': 8,
  'own_goals': 0,
  'pena

In [26]:
elements_df = pd.DataFrame(elements_list)

In [27]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,80201,0,0,-3,3,0,1,1.0,0.5,0,Bernd,0.0,1,False,,None,47,80201.jpg,1.3,Leno,1.1,False,None,a,1,3,4,57459,592,162893,3088,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,246,23,559,58,546,55,330,23,NaN,,NaN,,NaN,
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,2,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,0.8,False,None,u,1,3,0,19017,0,60021,860,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,436,40,418,18,382,13,441,40,NaN,,NaN,,NaN,
2,0.0,0.0,47431,0,0,-2,2,0,3,0.0,0.0,0,Willian,0.0,3,False,Transferred to Corinthians,2021-08-20T09:30:14.065783Z,63,47431.jpg,0.0,Borges Da Silva,0.1,False,None,u,1,3,0,914,0,20019,37,0.0,0.0,Willian,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,618,251,618,251,618,251,618,251,NaN,,NaN,,NaN,
3,100.0,100.0,54694,0,0,0,0,0,4,6.7,6.2,2,Pierre-Emerick,5.7,4,False,,2021-08-13T14:30:14.477664Z,100,54694.jpg,4.3,Aubameyang,5.3,False,None,a,1,3,39,543497,49712,343201,14628,0.6,3.9,Aubameyang,698,4,1,4,8,0,0,1,1,0,0,7,139,198.6,112.1,408.0,71.8,57,6,95,13,5,2,22,6,NaN,,NaN,,1.0,
4,NaN,NaN,58822,0,0,-3,3,0,2,1.0,0.5,0,Cédric,0.0,5,False,,None,42,58822.jpg,0.3,Soares,0.2,False,None,a,1,3,1,10689,193,20685,358,0.0,0.2,Cédric,188,0,0,0,7,0,0,0,1,0,0,0,21,30.8,36.1,1.0,6.8,328,126,230,63,360,128,341,121,NaN,,NaN,,NaN,


Stauts Field:  
s = suspended  
i = injury (major)  
u = transferred  
d = injury (75% chance of playing)  
n = not included in squad for some reason  
a = normal  

In [29]:
PLAYER_INFO = ['web_name', 'id', 'first_name', 'second_name', 'team', 'team_code', 'now_cost', 'cost_change_start', 'selected_by_percent']

In [30]:
FORM_INFO = [
    'form', 'points_per_game', 'total_points', 'value_form', 'value_season', 'minutes',
    'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves',
    'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
    'penalties_missed', 'penalties_saved'
]

ADDITIONAL_FORM_INFO = [
    'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 
    'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type',
    'own_goals', 'yellow_cards', 'red_cards', 
    'transfers_in_event', 'transfers_out_event'
]

In [31]:
elements_df_subset = elements_df[elements_df['status'].isin(['a', 'd'])].copy().reset_index(drop=True)
elements_df_subset = elements_df_subset[
    PLAYER_INFO + FORM_INFO + ADDITIONAL_FORM_INFO
].copy()

In [32]:
elements_df_subset['calculated_at_gw'] = next_gameweek - 1
elements_df_subset['season'] = CURRENT_SEASON

In [33]:
elements_df_subset.head()

,web_name,id,first_name,second_name,team,team_code,now_cost,cost_change_start,selected_by_percent,form,points_per_game,total_points,value_form,value_season,minutes,goals_scored,assists,clean_sheets,goals_conceded,saves,bonus,bps,influence,creativity,threat,ict_index,penalties_missed,penalties_saved,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,own_goals,yellow_cards,red_cards,transfers_in_event,transfers_out_event,calculated_at_gw,season
0,Leno,1,Bernd,Leno,1,3,47,-3,1.1,0.0,1.3,4,0.0,0.9,270,0,0,0,9,9,0,48,79.0,0.0,0.0,7.9,0,0,246,23,559,58,546,55,330,23,0,0,0,592,3088,10,2021/22
1,Aubameyang,4,Pierre-Emerick,Aubameyang,1,3,100,0,5.3,5.7,4.3,39,0.6,3.9,698,4,1,4,8,0,7,139,198.6,112.1,408.0,71.8,1,0,57,6,95,13,5,2,22,6,0,1,0,49712,14628,10,2021/22
2,Cédric,5,Cédric,Soares,1,3,42,-3,0.2,0.0,0.3,1,0.0,0.2,188,0,0,0,7,0,0,21,30.8,36.1,1.0,6.8,0,0,328,126,230,63,360,128,341,121,0,1,0,193,358,10,2021/22
3,Lacazette,6,Alexandre,Lacazette,1,3,83,-2,1.1,2.7,2.0,10,0.3,1.2,196,1,0,1,2,0,0,38,49.8,61.1,80.0,19.0,0,0,297,37,170,30,139,42,235,38,0,0,0,3555,2069,10,2021/22
4,Marí,8,Pablo,Marí,1,3,43,-2,0.1,0.0,0.5,1,0.0,0.2,180,0,0,0,4,0,0,35,55.4,12.3,11.0,7.9,0,0,280,110,315,108,310,101,329,116,0,1,0,5,233,10,2021/22


In [23]:
if SAVE_TO_DB_FLAG:
    elements_df_subset.to_sql('cummumative_player_info', db_conn, if_exists='append', index=False)

### Get Historical Player Stats
Historical Stats of Past Seasons for All Players  
`Note: Need to run this only at the begining of the season`

In [36]:
def historical_player_stats_helper(url):
    '''
    helper function for async requests for fetching historical player stats
    
    Inputs:
        - url: url string
        
    Returns:
        - hisorical_player_info: dataframe with player stats
    '''
    try:
        historical_player_info = rq.request('GET', url[0])
    except:
        return None
        
    historical_player_info = json.loads(historical_player_info.text)
    historical_player_info = pd.DataFrame(historical_player_info['history_past'])
    historical_player_info['player_id'] = url[1]
    return historical_player_info


if NEW_SEASON:
    player_ids_main_list = elements_df_subset['id'].unique()
    subset_size = 50

    player_ids_main_list_sublist = []

    for i in range(1, int(np.ceil(len(player_ids_main_list)/subset_size))+1):
        player_ids_main_list_sublist.append(player_ids_main_list[subset_size * (i-1) : subset_size * i])

    historical_player_results_list = []

    for player_id_list in tqdm(player_ids_main_list_sublist):
        url_list = [
            ('https://fantasy.premierleague.com/api/element-summary/' + str(player_id) + '/', player_id) for player_id in player_id_list
        ]

        #print(url_list)

        ### asyncio calls - fetch data for multiple urls parallely for faster processing
        event_loop = asyncio.get_event_loop()
        Executor = ThreadPoolExecutor(max_workers=len(url_list))
        tasks = [event_loop.run_in_executor(Executor, historical_player_stats_helper, url) for url in url_list]

        historical_player_results = await asyncio.gather(*tasks)
        historical_player_results_list.extend(historical_player_results)

    historical_stats_player_df = pd.concat(historical_player_results_list)
    try:
        existing_info = pd.read_sql_query('select * from historical_seasons_player_stats', db_conn)
        historical_stats_player_df = pd.concat([historical_stats_player_df, existing_info]).drop_duplicates().reset_index(drop=True)
    except Exception as e:
        print(str(e))
        
    historical_stats_player_df.to_sql('historical_seasons_player_stats', db_conn, if_exists='replace', index=False)

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]


### Authenticate and Fetch My Team

- Perform Authentication
- Fetch My Current Team Information, Available Chips and Transfers

In [37]:
sessions = rq.session()

url = 'https://users.premierleague.com/accounts/login/'

login_details = {
    'login': os.environ['USERNAME'],
    'password': os.environ['PASSWORD'],
    'redirect_uri': 'https://fantasy.premierleague.com/a/login',
    'app': 'plfpl-web'
}

sessions.post(url, data=login_details)

<Response [200]>

In [38]:
my_team_url = 'https://fantasy.premierleague.com/api/my-team/' + str(os.environ['TEAM_ID']) + '/'

my_team_response = sessions.get(my_team_url)
my_team_response = json.loads(my_team_response.text)

In [39]:
my_team = pd.DataFrame(my_team_response['picks'])
my_chips = pd.DataFrame(my_team_response['chips'])
my_transfers = pd.DataFrame([my_team_response['transfers']])

In [40]:
my_team['caclulated_at_gw'] = next_gameweek - 1
my_chips['caclulated_at_gw'] = next_gameweek - 1
my_transfers['caclulated_at_gw'] = next_gameweek - 1

my_team['season'] = CURRENT_SEASON
my_chips['season'] = CURRENT_SEASON
my_transfers['season'] = CURRENT_SEASON

my_chips['played_by_entry'] = my_chips['played_by_entry'].apply(lambda x: ' - '.join(x))

In [41]:
print(my_team.to_string(), end="\n\n")

print(my_chips.to_string(), end="\n\n")

print(my_transfers.to_string(), end="\n\n")

    element  position  selling_price  multiplier  purchase_price  is_captain  is_vice_captain  caclulated_at_gw   season
0        69         1             45           1              45       False            False                10  2021/22
1       237         2             75           1              75       False            False                10  2021/22
2       208         3             39           1              40       False            False                10  2021/22
3       185         4             43           1              45       False            False                10  2021/22
4       277         5            117           2             120        True            False                10  2021/22
5       254         6             87           1              90       False            False                10  2021/22
6       233         7            127           1             125       False             True                10  2021/22
7       188         8           

In [24]:
if SAVE_TO_DB_FLAG:
    my_team.to_sql('my_team_info', db_conn, if_exists='append', index=False)
    my_chips.to_sql('latest_chips_info', db_conn, if_exists='replace', index=False)
    my_transfers.to_sql('latest_transfer_info', db_conn, if_exists='replace', index=False)